**Effectuer les traitements nécessaires pour les variables catégorielles**   
❒ Choisir l'encodage pertinent (one hot encoding, label encoding, autre)   
❒ Filtrer les catégories présentes sous un certain seuil dans le dataset pour réduire la dimension   

**Mettre en oeuvre des techniques de réduction de dimension si nécessaire**   
❒ Appliquer une technique de réduction de dimension si besoin (exemple : Ne pas faire de PCA (ou autres) pour réduire la dimensions si les genres seulement sont binairisés)   

**Utiliser un modèle d'apprentissage non supervisé approprié**   
❒ Utiliser une distance pertinente en fonction de la nature des données, éventuellement plusieurs distances moyennées pour différents groupes de variables si elles ne sont pas du même type   
❒ si un algorithme de clustering est utilisé, le paramétriser de façon efficace (si kmeans, k doit être assez grand (pas 3), mais pas trop (clusters avec moins de 5 éléments)), ou hierarchical clustering pour avoir des granularités variables 
❒ Ou tester de prendre les N films les plus proches   
❒ essayer à la fois une approche de clustering et une approche de voisins, discuter des avantages et incovénients de chaque méthode dans le cadre du projet   

**Évaluer les performances d’un modèle d’apprentissage non supervisé**   
❒ Si critère basé sur des comparaisons de distance intra/inter clusters, ne pas comparer des distances incomparables (pas définies sur le même set de variables, dans la même dimension...)   
❒ Avoir une approche empirique raisonnable : prendre des films variés pour avoir une vision globale   
❒ Essayer d'élaborer les appréciations des performances (ne pas se contenter d'un commentaire simpliste)   
❒ réfléchir à ce qu'est une bonne recommandation : a-t-on besoin de ML pour regarder harry potter 4 après le 3   
❒ essayer de quanitifer les performances (réfléchir à, voire mettre en oeuvre un système de notation des différentes approches)   

In [1]:
import numpy as np
import pandas as pd
import scipy
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import ParameterGrid
from sklearn import metrics
from sklearn.metrics import silhouette_score
import calendar;
import time;
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from IPython.display import Audio
sound_file = 'rossignol.wav'

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('df_cleaned.csv', index_col = 'index')

In [3]:
df.shape

(4917, 28)

In [4]:
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
index,,,,,,,,,,,,,,,,,,,,,
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
2705,Color,Ron Shelton,71.0,118.0,41.0,274.0,Dash Mihok,556.0,9059588.0,Crime|Drama|Romance|Thriller,...,125.0,English,USA,R,15000000.0,2002.0,463.0,6.6,2.35,455
2706,Color,Michael Winterbottom,190.0,108.0,187.0,254.0,Archie Panjabi,11000.0,9172810.0,Biography|Drama|History|Thriller|War,...,118.0,English,USA,R,16000000.0,2007.0,883.0,6.7,2.35,923
2707,Color,David Raynr,50.0,94.0,9.0,612.0,Jodi Lyn O'Keefe,11000.0,8735529.0,Comedy|Drama|Romance,...,89.0,English,USA,PG-13,15000000.0,2000.0,897.0,5.5,1.85,816
2708,Color,Mort Nathan,63.0,97.0,2.0,461.0,Lin Shaye,890.0,8586376.0,Comedy,...,132.0,English,USA,R,20000000.0,2002.0,852.0,4.9,1.85,1000


## 1. Effectuer les traitements nécessaires pour les variables catégorielles

### 1.1. Choisir l'encodage pertinent (one hot encoding, label encoding, autre)

Pour director_name je vais utiliser OneHotEncoder
Pour les catégories à plusieurs labels séparés par un |, je vais utiliser TfidfVectorizer parce qu'il me permet d'utiliser une fonction tokenize qui sait isoler les différentes catégories séparées par un |.
avec use_idf = False pour rapporter la fréquence du terme au film seulement (3 termes = fréquence de 0,33 chacun) et non pas à l'ensemble des films.   

je préfère cet encoding à l'encoding LabelEncoder qui introduit des proximités artificielles.

In [5]:
def split_tokenizer(s):
   return s.split('|')

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(tokenizer = split_tokenizer, use_idf = False)

from sklearn.preprocessing import OneHotEncoder
one_hot = OneHotEncoder()


### 1.2. Filtrer les catégories présentes sous un certain seuil dans le dataset pour réduire la dimension

Avant de transformer les variables catégorielles en dummy colonnes, je réduit le nombre de colonnes résultats en ne conservant que les catégories les plus fréquentes (eg.qui apparaissent pour au moins 6 films) pour director_name, actor_n_name et plot_keywords.    
Je définis donc des fonctions helper pour ces traiments

In [6]:
# J'ai recopié cette fonction trouvée dans un ce kernel Kaggle : https://www.kaggle.com/fabiendaniel/film-recommendation-engine
# car la trouve très pratique. Comme elle est codée très compacte, je l'ai documentée pour faciliter sa compréhension.

def count_word(df, ref_col, liste):

#paramètres :
#df : la dataframe
#col : la colonne
#liste : vocabulaire complet (ou restreint) pour la colonne considérée = liste des valeurs uniques prises par la colonne, 
#après séparation par split "|"

#variables intermédiaires
#liste_words : liste des mots trouvés dans une colonne après séparation par '|'

    #initialisation d'un dictionnaire compteur avec une entrée par mot du vocabulaire
    # ce dictionnaire contiendra le nb d'occurences du mot dans la dataframe
    word_count = dict()
    for s in liste: word_count[s] = 0
    
    # liste des mots dans la colonne
    for liste_words in df[ref_col].str.split('|'):        
        
        # si la valeur dans la colonne n'est pas du texte (image_ratio par exemple)
        if type(liste_words) == float and pd.isnull(liste_words): continue   #pas compris "and pd.isnull(liste_words)"
        # si un mot de la colonne est présent dans la liste entrée en paramètre de la fonction, on augmente le compteur
        for s in [s for s in liste_words if s in liste]: 
            if pd.notnull(s): word_count[s] += 1
    #______________________________________________________________________
    # convert the dictionary in a list to sort the keywords by frequency
    word_occurences = []
    for w,v in word_count.items():
        word_occurences.append([w,v])
    word_occurences.sort(key = lambda x:x[1], reverse = True)
    return word_occurences, word_count


In [7]:
# Cette fonction retourne la liste des valeurs d'une catégorie utilisées au moins 6 fois dans le dataset
# le nombre de 6 est relativement arbitraire car on pourrait recommander 4 films avec un directeur commun 
# et 2 films avec un acteur commun. 

def frequent_categories(df, column, n=0) :

    category_set = df[column].unique()
    category_occurences, dum = count_word(df, column, category_set)
    df_category_occurences = pd.DataFrame(category_occurences, columns = ['word', 'count'])
    frequent_categories = df_category_occurences[df_category_occurences['count'] > n ]['word'].unique()
    return list(frequent_categories)

### 2.1 Directors

In [8]:
def pre_process_director(n_frequency=0) :
    
    frequent_directors = frequent_categories(df, 'director_name', n_frequency)
    frequent_directors
    
    # **Je remplace les directeurs peu fréquents par la valeur None**    
    df["director_name"] = df["director_name"].apply(
        lambda x: x  if x in (frequent_directors) else None)
    
    #**J'encode avec OneHot**
    encoded_director =  one_hot.fit_transform(df[["director_name"]].values.astype('U'))
    return encoded_director

In [9]:
encoded_director = pre_process_director()
encoded_director

<4917x2398 sparse matrix of type '<class 'numpy.float64'>'
	with 4917 stored elements in Compressed Sparse Row format>

### 2.2 acteurs
**Je vais regrouper les acteurs pour éviter qu'un fim B soit considéré comme non similaire à un film A parce que Julia Roberts est en acteur 2 dans l'un et en acteur 1 dans l'autre**

**auparavant je ne conserve que  les acteurs cités dans au moins 6 films, soit comme acteur 1, soit acteur 2, soit acteur 3**

In [10]:
def pre_process_actors(n_frequency=0) :
# liste globale des acteurs cités au moins n fois
    frequent_actors = set()
    frequent_actor_1_name = frequent_categories(df, 'actor_1_name',n_frequency)
    frequent_actors.update(frequent_actor_1_name)
    frequent_actor_2_name = frequent_categories(df, 'actor_2_name', n_frequency)
    frequent_actors.update(frequent_actor_2_name)
    frequent_actor_3_name = frequent_categories(df, 'actor_3_name', n_frequency)
    frequent_actors.update(frequent_actor_3_name)
    frequent_actors = set(frequent_actors)

    #Je remplace les valeurs rares par None
    df["actor_1_name"] = df["actor_1_name"].apply(
        lambda x: x  if x in (frequent_actors) else '')
    df["actor_2_name"] = df["actor_2_name"].apply(
        lambda x: x  if x in (frequent_actors) else '')
    df["actor_3_name"] = df["actor_3_name"].apply(
        lambda x: x  if x in (frequent_actors) else '')
    
    # Je regroupe les acteurs
    df['actors'] = df['actor_1_name'] + '|' + df['actor_2_name']  + '|' + df['actor_3_name']
    
    # J'encode avec TFiDF
    encoded_actors = tfidf_vectorizer.fit_transform(df['actors'].values.astype('U'))
    
    return encoded_actors

In [11]:
encoded_actors = pre_process_actors()
encoded_actors

<4917x6252 sparse matrix of type '<class 'numpy.float64'>'
	with 14731 stored elements in Compressed Sparse Row format>

### 2.3 Plot_keywords
**Je vais encoder par Tfidf seulement les mots clés utilisés pour au moins n films**

In [12]:
def pre_process_keywords(n_frequency=0) :
    # Calcul des occurrence des mots clés
    keywords_set = set()

    for string in df[df.plot_keywords.notnull()]['plot_keywords'].str.split('|').values:
        keywords_set = keywords_set.union(set(string))
    
    keyword_occurences, dum = count_word(df, 'plot_keywords', keywords_set)
    # identification des mots clés utilisés moins de 6 fois, pour les exclure de l'encodinf

    df_keyword_occurences = pd.DataFrame(keyword_occurences, columns = ['word', 'count'])
    stop_keywords = list(df_keyword_occurences[df_keyword_occurences['count'] <n_frequency ]['word'])
    #stop_keywords
    
    # J'encode avec Tfidf
    tfidf_vectorizer = TfidfVectorizer(tokenizer = split_tokenizer, stop_words = stop_keywords, use_idf = False)
    encoded_keywords = tfidf_vectorizer.fit_transform(df['plot_keywords'].values.astype('U'))
    
    return encoded_keywords

In [13]:
encoded_keywords = pre_process_keywords()
encoded_keywords

<4917x8086 sparse matrix of type '<class 'numpy.float64'>'
	with 23639 stored elements in Compressed Sparse Row format>

**On pourrait réduire ce nombre en utilisant une lemmisation (mots partageant une même racine) ou un word-embedding (réduction de dimension grâce à une réprésenttaion sémentique du vocabulaire)** Je ne l'ai pas fait pour ce projet.

### 2.4 Genres
**Je vais encoder en Tfidf les genres cités pour au moins n films**

In [14]:
def pre_process_genres(n_frequency=0) :
    genres_set = set()

    for string in df[df.genres.notnull()]['genres'].str.split('|').values:
        genres_set = genres_set.union(set(string))
    
    genres_occurences, dum = count_word(df, 'genres', genres_set)
    # identification des mots clés utilisés moins de n fois, pour les exclure de l'encodinf

    df_genres_occurences = pd.DataFrame(genres_occurences, columns = ['word', 'count'])
    stop_genres = list(df_genres_occurences[df_genres_occurences['count'] <n_frequency ]['word'])
    #stop_keywords
    
    # J'encode avec Tfidf
    tfidf_vectorizer = TfidfVectorizer(tokenizer = split_tokenizer, stop_words = stop_genres, use_idf = False)
    encoded_genres = tfidf_vectorizer.fit_transform(df['genres'].values.astype('U'))
    
    return encoded_genres



In [15]:
encoded_genre = pre_process_genres()
encoded_genre

<4917x26 sparse matrix of type '<class 'numpy.float64'>'
	with 14126 stored elements in Compressed Sparse Row format>

### 2.5 content-rating

**Je fais un encodage manuel ordonné de ma jière croissante du film grand public jusqu'au film le plus restreint**
J'ai regardé les définitions des content-rating sur wikipedia, et j'en ai conclu qu'ils pouvaient être ordonnés de la manière suivante :

In [16]:
 
# J'encode avec un encodage manuel
def content_rating_encoded(label) :
    if label in('G', 'T-Y', 'TV-G', 'Unrated', 'Non rated' , 'passed', 'approved', 'rated', 'non rated') : 
        x=int(1)
    elif label in ('PG', 'GP') :  
        x = int(2)
    elif label in ('PG-13', 'TV-14') :
        x = int(3)
    elif label in ('R', 'X') :
        x = int(4)
    elif label in('NC-17', 'TV-MA') :
        x = int(5)
    else : 
        x = int(1)
        
    return x

def pre_process_content_rating(n_frequency=0) :
    
    df["content_rating"] = df["content_rating"].apply(
        lambda x: content_rating_encoded(x))
    
    return df["content_rating"]


In [17]:
df["content_rating"] = pre_process_content_rating()
#df["content_rating"]

In [18]:
encoded_content_rating = pre_process_content_rating()

In [19]:
#encoded_content_rating

### 2.6 language, country

**Je vais réaliser un encoding One Hot en ne conservant que les pays et langauges utilisés dans au moins 6 films**

In [20]:
def pre_process_country(n_frequency=0) :
    frequent_countries = frequent_categories(df, 'country', n_frequency)
    df["country"] = df["country"].apply(
        lambda x: x  if x in (frequent_countries) else None)
    encoded_country =  one_hot.fit_transform(df[['country']].values.astype('U'))
    return encoded_country

In [21]:
encoded_country = pre_process_country()
encoded_country

<4917x65 sparse matrix of type '<class 'numpy.float64'>'
	with 4917 stored elements in Compressed Sparse Row format>

In [22]:
def pre_process_language(n_frequency=0) :
    frequent_languages = frequent_categories(df, 'language', n_frequency)
    df["language"] = df["language"].apply(
        lambda x: x  if x in (frequent_languages) else None)
    encoded_language =  one_hot.fit_transform(df[['language']].values.astype('U'))
    return encoded_language

In [23]:
encoded_language = pre_process_language()
encoded_language

<4917x47 sparse matrix of type '<class 'numpy.float64'>'
	with 4917 stored elements in Compressed Sparse Row format>

### 2.6 Normalisation des valeurs numériques

Je vais normaliser entre 0 et 1, qui est l'intervalle dans lequel se trouvent déjà les valeurs des dummy variables catégorielles.
Dois-je standardiser (centrer-réduire) ? Si j'ai bien compris, je pense que non. En effet à part la note Imdb, toutes ces variables ont des distributions très différentes d'une distribution gaussienne. Je pense que la standardisation transformerait trop les données.

Avant de normaliser je vais traiter les outliers. J'en ai trouvé 1 : le budget le plus élevé, qui n'est pas une valeur aberrante mais qui fausserait le résultat de la normalisation MinMax.

Je vais remplacer cette valeur par une valeur légèrement au dessus (+ 10%) de la 2ème valeur la plus forte.


In [24]:
max1_budget = df.sort_values(by = 'budget', ascending = False)['budget'].values[0]
max2_budget = df.sort_values(by = 'budget', ascending = False)['budget'].values[1]
mask = df.budget == max1_budget
column_name = 'budget'
df.loc[mask, column_name] = max2_budget * 1.1

#df.loc[[2988]]


In [25]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

def pre_process_numeric_variables() :
    
    numeric_columns = [
    'duration', 'director_facebook_likes', 'actor_3_facebook_likes', 'actor_1_facebook_likes', \
    'gross', 'num_voted_users', 'cast_total_facebook_likes', 'facenumber_in_poster', \
    'num_user_for_reviews','budget', 'title_year', 'actor_2_facebook_likes', 'imdb_score', \
    'movie_facebook_likes', 'content_rating', 'aspect_ratio'
    ]

    scaler = MinMaxScaler(feature_range=(0, 1))
    df[numeric_columns] = pd.DataFrame(scaler.fit_transform(df[numeric_columns]))
    
    #l'encodage MinMax des vaeurs numériques a produit des valeurs nan que je remplace par la valeur moyenne de la colonne
    
    imp_median = SimpleImputer(strategy='median')

    df[numeric_columns] = imp_median.fit_transform(df[numeric_columns])
    df_numeric = df[numeric_columns].to_sparse(fill_value=0)
    
    return df_numeric


In [26]:
df_numeric = pre_process_numeric_variables()
df_numeric.head()

,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,movie_facebook_likes,content_rating,aspect_ratio
index,,,,,,,,,,,,,,,,
0,0.339286,0.000000,0.037174,0.001563,1.000000,0.524453,0.007361,0.000000,0.603479,0.051299,0.93,0.006832,0.797468,0.094556,0.0,0.040486
2705,0.309524,0.608696,0.017913,0.001164,0.062299,0.104708,0.003851,0.000000,0.162680,0.015151,0.89,0.003547,0.759494,0.000000,0.0,0.078947
2706,0.267857,0.036739,0.034217,0.020313,0.057103,0.055327,0.025246,0.023256,0.077683,0.015151,0.87,0.007299,0.632911,0.000000,0.0,0.078947
2707,0.154762,0.003652,0.025870,0.001563,0.189553,0.043993,0.006344,0.000000,0.019174,0.015476,0.82,0.007299,0.468354,0.001989,0.0,0.045209
2708,0.196429,0.000000,0.010522,0.001109,0.299755,0.160725,0.002551,0.000000,0.461356,0.015584,0.86,0.003861,0.645570,0.000000,0.0,0.045209


### Jeu de test

Je place dans mon jeu d'essai :

2 films grand succès dans 2 genres différents : star wars et notting hill   
un documentaire   
un film en japonais

In [27]:
user_films = [236, 1155, 2323, 3770]

**Je définis une fonction helper pour l'affichage des résultats de recommandation à partir du jeu d'essai**

In [28]:
from IPython.display import display, HTML

# reçoit un array en entrée
def print_films_by_position (films_position) :
    df_films = df.iloc[films_position]
    columns = ['movie_title', 'genres', 'budget', 'content_rating', \
               'director_name', 'actors', 'plot_keywords', 'country', 'language', 'title_year']
    display(HTML(df_films[columns].to_html()))
    
def print_films_by_index (films_index) :
    df_films = df.loc[films_index]
    columns = ['movie_title', 'genres', 'budget', 'content_rating', \
               'director_name', 'actors', 'plot_keywords', 'country', 'language', 'title_year']
    display(HTML(df_films[columns].to_html()))
    
def knn_results_positions(film_index) :
    film_position = df.index.get_loc(film_index)
    recommended_films_positions = indices[film_position].tolist()
    return recommended_films_positions

    
def print_knn_results_essai(user_films_index) :
    for film_index in user_films_index :   
        film_position = df.index.get_loc(film_index)
        print ("user film : ")
        print_films_by_position([film_position])
        recommended_films_positions = knn_results_positions(film_index)
        print ("recommended films : ")
        print_films_by_position(recommended_films_positions)

        
  
    

In [29]:
def clustering_results_index(film_index) :
    film_position = df.index.get_loc(film_index)
    film_cluster = labels[film_position]
    recommended_films_positions = ([i for i in range(len(labels)) if labels[i] == film_cluster])
    recommended_films_index = df.iloc[recommended_films_positions[:5]].index
    return recommended_films_index  

In [30]:
def print_clustering_results_essai(user_films_index) :
    for film_index in user_films_index :   
        film_position = df.index.get_loc(film_index)
        print ("user film : ")
        print_films_by_position([film_position])
        recommended_films_index = clustering_results_index(film_index)
        print ("recommended films : ")
        print_films_by_index(recommended_films_index)

## Sélection de variables pour la modélisation

In [31]:
import scipy

X = scipy.sparse.hstack([encoded_director,encoded_actors, encoded_genre, encoded_country, encoded_language, df_numeric]).todense()

X_df = pd.DataFrame(X)
X_df.to_csv('X_df_sans_keywords_nb_variables_' + str(X_df.shape[1]) + '.csv')
X_df.shape


(4917, 8804)

## Réduction de dimension

In [32]:


n_components_parameters = [500, 1000, 1500, 2000, 5000, 6000, 7000, 8000, 9000, 10000]

svd_explained_variance_ratio = []

for n_components in n_components_parameters :

    svd = TruncatedSVD(n_components=n_components, n_iter=7, random_state=42)
    svd.fit(X)  
    svd_explained_variance_ratio.append(svd.explained_variance_ratio_.sum())


ValueError: n_components must be < n_features; got 9000 >= 8804

In [ ]:
ts = calendar.timegm(time.gmtime())

import simplejson
f = open('svd_explained_variance_ratio_sans_keywords_pas_de_filtre' + str(ts), 'w')
simplejson.dump(svd_explained_variance_ratio, f)
f.close()

In [ ]:
fig, ax = plt.subplots(figsize = (7,7))

ax = sns.lineplot(n_components_parameters, svd_explained_variance_ratio, color = 'b')
ax.set_title('variance expliquée cumulée')
ax.set_ylabel('variance expliquée cumulée')
ax.set_xlabel('nb dimensions')



## Modèle 1 : Nearest Neighbours

In [ ]:
from sklearn.neighbors import NearestNeighbors

ts = calendar.timegm(time.gmtime())

nbrs = NearestNeighbors(n_neighbors=6).fit(X)
distances, indices = nbrs.kneighbors(X_df)
np.save(indices, 'knn_baseline_indices_sans_keywords_sans_filtre' + str(ts))

### - Test Recommandation, cas Nearest Neighbors

In [ ]:
print_knn_results_essai(user_films_index)

## Modèle 2 : K means

### - Modélisation

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

km = KMeans(n_clusters=100)
km_labels = km.fit_predict(X)

In [ ]:
import seaborn as sns

ax = sns.distplot(labels)

### - Test des recommandations

In [ ]:
print_clustering_results_essai(user_films_index)


### - Optimisation

In [ ]:
# The Silhouette Coefficient is calculated using the mean intra-cluster distance (a) 
# and the mean nearest-cluster distance (b) for each sample. 
# Silhouette Coefficient for a sample is (b - a) / max(a, b).

#The best value is 1 and the worst value is -1. 
# Values near 0 indicate overlapping clusters. 
# Negative values generally indicate that a sample has been assigned to the wrong cluster, as a different cluster is more similar.

# http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py


In [ ]:
from sklearn.model_selection import ParameterGrid
from sklearn import metrics
from sklearn.metrics import silhouette_score

import calendar;
import time;
ts = calendar.timegm(time.gmtime())
print(ts)
# 1540767599

grid = [{'p1_n_frequency': [1,2,6], 'p2_n_clusters': [100, 200, 300,400], 'p3_model': ['km','ag']}]
#grid = [{'p1_n_frequency': [1, 6, 30], 'p2_n_clusters': [100, 300, 400, 1000], 'p3_model': ['km'], 'p4_svd : []'}]

warnings.filterwarnings('ignore')

df_initial = pd.read_csv('df_cleaned.csv', index_col = 'index')

estimator_scores = []
nb_features = []
sizes = []
for g in ParameterGrid(grid) :
    df = df_initial.copy()
    model = g['p3_model']
    if model == 'km' :
        estimator = KMeans()
    elif model == 'ag' :
        estimator = AgglomerativeClustering()
    n_clusters = g['p2_n_clusters']
    estimator.set_params(**{'n_clusters': n_clusters})
    n_frequency = g['p1_n_frequency']
    encoded_actors = pre_process_actors(n_frequency)
    encoded_director = pre_process_director(n_frequency)
    encoded_keywords = pre_process_keywords(n_frequency)    
    encoded_country = pre_process_country(n_frequency)
    encoded_language = pre_process_language(n_frequency)
    pre_process_content_rating(n_frequency)
    df_numeric = pre_process_numeric_variables()
    X = scipy.sparse.hstack([encoded_director,encoded_actors, encoded_genre, encoded_country, encoded_language]).todense()
    estimator.fit(X)
    estimator_labels = estimator.labels_
    estimator_score = silhouette_score(X, estimator_labels)
    estimator_scores.append([model, n_clusters, n_frequency, estimator_score])
    nb_features.append(df.shape[1])
    cluster_labels = np.unique(estimator.labels_)
    for cluster in cluster_labels :
        count = len(estimator_labels[estimator_labels == cluster])
        sizes.append([model, n_clusters, n_frequency, count])

    df.assign(label = estimator_labels)
    filename = 'df_clustering_results_sans_coeff_sans_keywords_' + str(n_frequency) + "_" + str(n_clusters) + "_" + str(model) + ".csv"
    df.to_csv(filename, index_label = 'id')
    print ('n_frequency : ' + str(n_frequency) + ', n_clusters : ' + str(n_clusters) + ' , model : ' + str(model)  + ' - nb de features = ' + str(X.shape[1]) +  ' - silhouette_score = ' + str(estimator_score))
    
np.save('estimator_scores' + '_' + str(ts), estimator_scores)
np.save('nb_features'+ '_' + str(ts), nb_features)
np.save('sizes'+ '_' + str(ts), sizes)

In [ ]:
sizes = np.load('sizes_1541892709.npy')
df_boxplot = pd.DataFrame(sizes, columns = ['model', 'n_clusters', 'n_frequency','size'])
df_boxplot = df_boxplot[df_boxplot['n_frequency'] == '6']
df_boxplot = df_boxplot.drop(columns = ['n_frequency'])
df_boxplot['n_clusters'] = df_boxplot['n_clusters'].apply( lambda x : int(x))
df_boxplot['size'] = df_boxplot['size'].apply(lambda x: int(x))

sizes = np.load('estimator_scores_1541892709.npy')
df_lineplot = pd.DataFrame(sizes, columns = ['model', 'n_clusters', 'n_frequency','score'])
df_lineplot = df_lineplot[df_lineplot['n_frequency'] == '6']
df_lineplot = df_lineplot.drop(columns = ['n_frequency'])
df_km_lineplot = df_lineplot[df_lineplot['model'] == 'km']
df_ag_lineplot = df_lineplot[df_lineplot['model'] == 'ag']
parameters = df_ag_lineplot['n_clusters'].values.astype(np.float)
km_scores = df_km_lineplot ['score'].values.astype(np.float)
ag_scores = df_ag_lineplot['score'].values.astype(np.float)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax =plt.subplots(1,2, figsize=(20,10))
ax[0].set_title('Coefficient Silhouette', fontsize=14)
ax[0].set_ylabel('Coefficient Silhouette', fontsize=14)
ax[0].set_xlabel('nb de clusters', fontsize=14)
ax[1].set_title('Taille des clusters', fontsize=14)
ax[1].set_ylabel('Taille des clusters', fontsize=14)
ax[1].set_xlabel('nb de clusters', fontsize=14)

sns.lineplot(parameters, km_scores, color = 'b', ax=ax[0], ci=None)
sns.lineplot(parameters, ag_scores, color = 'orange', ax=ax[0], ci=None)
sns.boxplot(y='size', x='n_clusters', hue='model',data=df_boxplot,  showfliers = False,  ax=ax[1], )
#ax[1].set_yscale("log")
#sns.boxplot(y='size', x='n_clusters', hue='model',data=df_boxplot, ax=ax[1])


**Avec le modèle Agglomerative clustering, le coefficient de silhouette est meilleur et la taille des clusters est plus homogène.**

**comparaison SVD et filtrage des valeurs**

In [ ]:
f_initial = pd.read_csv('df_cleaned.csv', index_col = 'index')

estimator_scores = []
nb_features = []
sizes = []
model == 'ag'
estimator = AgglomerativeClustering(n_clusters= 300
n_frequency = 0
encoded_actors = pre_process_actors(n_frequency)
encoded_director = pre_process_director(n_frequency)
encoded_keywords = pre_process_keywords(n_frequency)    
encoded_country = pre_process_country(n_frequency)
encoded_language = pre_process_language(n_frequency)
pre_process_content_rating(n_frequency)
df_numeric = pre_process_numeric_variables()
X = scipy.sparse.hstack([encoded_director,encoded_actors, encoded_genre, encoded_country, encoded_language]).todense()
estimator.fit(X)
estimator_labels = estimator.labels_
estimator_score = silhouette_score(X, estimator_labels)
estimator_scores.append([model, n_clusters, n_frequency, estimator_score])
nb_features.append(df.shape[1])
cluster_labels = np.unique(estimator.labels_)
for cluster in cluster_labels :
    count = len(estimator_labels[estimator_labels == cluster])
    sizes.append([model, n_clusters, n_frequency, count])

df.assign(label = estimator_labels)
filename = 'df_clustering_results_svd_' + str(n_frequency) + "_" + str(n_clusters) + "_" + str(model) + ".csv"
df.to_csv(filename, index_label = 'id')
print ('n_frequency : ' + str(n_frequency) + ', n_clusters : ' + str(n_clusters) + ' , model : ' + str(model)  + ' - nb de features = ' + str(X.shape[1]) +  ' - silhouette_score = ' + str(estimator_score))
    
np.save('estimator_scores_svd' + '_' + str(ts), estimator_scores)
np.save('nb_features_svd'+ '_' + str(ts), nb_features)
np.save('sizes_svd'+ '_' + str(ts), sizes)

## Modèle retenu
**Agglomerative clustering avec 300 clusters, après filtrage des valeurs utilisées pour au moins 6 films**

In [ ]:
df_initial = pd.read_csv('df_cleaned.csv', index_col = 'index')
df = df_initial.copy()

estimator_scores = []
nb_features = []
sizes = []
model == 'ag'
n_frequency = 0
n_clusters = 300
estimator = AgglomerativeClustering(n_clusters= n_clusters)
encoded_actors = pre_process_actors(n_frequency)
encoded_director = pre_process_director(n_frequency)
encoded_keywords = pre_process_keywords(n_frequency)    
encoded_country = pre_process_country(n_frequency)
encoded_language = pre_process_language(n_frequency)
pre_process_content_rating(n_frequency)
df_numeric = pre_process_numeric_variables()
X = scipy.sparse.hstack([encoded_director,encoded_actors, encoded_genre, encoded_country, encoded_language]).todense()
estimator.fit(X)
estimator_labels = estimator.labels_
estimator_score = silhouette_score(X, estimator_labels)
estimator_scores.append([model, n_clusters, n_frequency, estimator_score])
nb_features.append(df.shape[1])
cluster_labels = np.unique(estimator.labels_)
for cluster in cluster_labels :
    count = len(estimator_labels[estimator_labels == cluster])
    sizes.append([model, n_clusters, n_frequency, count])

df.assign(label = estimator_labels)
filename = 'df_clustering_results_sans_coeff_sans_keywords_' + str(n_frequency) + "_" + str(n_clusters) + "_" + str(model) + ".csv"
df.to_csv(filename, index_label = 'id')
print ('n_frequency : ' + str(n_frequency) + ', n_clusters : ' + str(n_clusters) + ' , model : ' + str(model)  + ' - nb de features = ' + str(X.shape[1]) +  ' - silhouette_score = ' + str(estimator_score))
    
np.save('estimator_scores' + '_' + str(ts), estimator_scores)
np.save('nb_features'+ '_' + str(ts), nb_features)
np.save('sizes'+ '_' + str(ts), sizes)

### Exemples de résultats

In [ ]:
user_films_index = [523,236, 1155, 2323, 1224, 344, 1065]
labels = estimator.labels_
print_clustering_results_essai(user_films_index)

## Mise au format API 

### - Enregistrement du résultat de la modélisation, cas clustering

In [ ]:
df.assign(label = ag_labels)
df.to_csv('df_clustering_results.csv')



### - Enregistrement du résultat de la modélisation, cas clustering

In [ ]:
df.assign(indices = indices)
df.to_csv('df_neighbors_results.csv')

In [ ]:
#fonction à recopier dans l'API

# récupérer les résultats de la modélisation
results = pd.read_csv('df_clustering_results.csv')

import simplejson as json

def knn_results_index(film_index) :
    film_position = df.index.get_loc(film_index)
    recommended_films_positions = indices[film_position].tolist()
    recommended_films_index = df.iloc[recommended_films_positions].index.tolist()
    return recommended_films_index

def clustering_results_index(film_index) :
    film_position = df.index.get_loc(film_index)
    film_cluster = df.iloc[[film_position]]['label']
    recommended_films = df[df['label'] == film_cluster]]
    recommended_films_index = recommended_films.index.tolist()
    return recommended_films_index  

# afficher les recommandations
def return_json_recommended_films(film_index) :
    film_index = int(film_index)
    # récupérer les résultats du modèle
    # si clustering
    recommended_films_index = clustering_results_index(film_index)
    # si knn
    #recommended_films_index = knn_results_index(film_index)
    
    results = []
    for index in recommended_films_index :
        dict_film = {'id' : index, 'name' : df.loc[[index]]['movie_title'].values[0].split("\xa0")[0] }
        results.append(dict_film)
    dict_results = {'_results': results }
    print(json.dumps(dict_results))

In [ ]:
#vérification de l'API
film_index = 2364
return_json_recommended_films(film_index)